# Reflection Service for Toxicity Reduction

**NOTE**: This is adapted from the original notebook in the [core llama-index repo](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/agent/llama-index-agent-introspective/examples/toxicity_reduction.ipynb).

In this notebook, we cover how to setup a reflection service that can perform toxicity reflection and correction.

We make use of two types of reflection services as "agents" in llama-agents: 

- A self-reflection agent that can reflect and correct a given response without any external tools
- A CRITIC agent that can reflect and correct a given response using external tools.

We set these up as **independent** services, meaning they don't communicate. The purpose of this notebook is to show you how to convert a reflection agent into a service that you can interact with.

In this notebook we make use of our prepackaged reflection agents using our `llama-index-agent-introspective` LlamaPack. This is primarily for concision.

*However*, if you wish to build reflection from scratch we highly encourage you to do so! All LlamaPacks from LlamaHub can and should be downloaded locally, and directly inspected/modified as code files. This is highly encouraged.

In [ ]:
%pip install llama-index-agent-introspective -q
%pip install google-api-python-client -q
%pip install llama-index-llms-openai -q
%pip install llama-index-program-openai -q
%pip install llama-index-readers-file -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

## 1 Toxicity Reduction: Problem Setup

In this notebook, the task we'll have our introspective agents perform is "toxicity reduction". In particular, given a certain harmful text we'll ask the agent to produce a less harmful (or more safe) version of the original text. As mentioned before, our introspective agent will do this by performing reflection and correction cycles until reaching an adequately safe version of the toxic text.

### 1.a Setup our CRITIC Agent

Our CRITIC Agent makes use of an external tool to reflect/validate the response, and then correct it. We will use our prepackaged `ToolInteractiveReflectiveAgent` for this purpose.

The CRITIC agent delegates the critique subtask to a `CritiqueAgentWorker`, and then performs correction with a standalone LLM call.

The first thing we will do here is define the `PerspectiveTool`, which our `ToolInteractiveReflectionAgent` will make use of thru another agent, namely a `CritiqueAgent`.

To use Perspecive's API, you will need to do the following steps:

1. Enable the Perspective API in your Google Cloud projects
2. Generate a new set of credentials (i.e. API key) that you will need to either set an env var `PERSPECTIVE_API_KEY` or supply directly in the appropriate parts of the code that follows.

To perform steps 1. and 2., you can follow the instructions outlined here: https://developers.perspectiveapi.com/s/docs-enable-the-api?language=en_US.

#### Build `PerspectiveTool`

In [ ]:
from googleapiclient import discovery
from typing import Dict, Optional
import json
import os


class Perspective:
    """Custom class to interact with Perspective API."""

    attributes = [
        "toxicity",
        "severe_toxicity",
        "identity_attack",
        "insult",
        "profanity",
        "threat",
        "sexually_explicit",
    ]

    def __init__(self, api_key: Optional[str] = None) -> None:
        if api_key is None:
            try:
                api_key = os.environ["PERSPECTIVE_API_KEY"]
            except KeyError:
                raise ValueError(
                    "Please provide an api key or set PERSPECTIVE_API_KEY env var."
                )

        self._client = discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=api_key,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        )

    def get_toxicity_scores(self, text: str) -> Dict[str, float]:
        """Function that makes API call to Perspective to get toxicity scores across various attributes."""

        analyze_request = {
            "comment": {"text": text},
            "requestedAttributes": {att.upper(): {} for att in self.attributes},
        }

        response = self._client.comments().analyze(body=analyze_request).execute()
        try:
            return {
                att: response["attributeScores"][att.upper()]["summaryScore"]["value"]
                for att in self.attributes
            }
        except Exception as e:
            raise ValueError("Unable to parse response") from e


perspective = Perspective()

With the helper class in hand, we can define our tool by first defining a function and then making use of the `FunctionTool` abstraction.

In [ ]:
from typing import Tuple
from llama_index.core.bridge.pydantic import Field


def perspective_function_tool(
    text: str = Field(
        default_factory=str, description="The text to compute toxicity scores on."
    )
) -> Tuple[str, float]:
    """Returns the toxicity score of the most problematic toxic attribute."""

    scores = perspective.get_toxicity_scores(text=text)
    max_key = max(scores, key=scores.get)
    return (max_key, scores[max_key] * 100)


from llama_index.core.tools import FunctionTool

pespective_tool = FunctionTool.from_defaults(
    perspective_function_tool,
)

A simple test of our perspective tool!

In [ ]:
perspective_function_tool(text="friendly greetings from python")

('toxicity', 2.5438840000000003)

#### Build a stateful agent function with `ToolInteractiveReflectionAgent`

We define a stateful agent function that wraps the prepackaged `ToolInteractiveReflectionAgent`. This stateful agent function will then be directly turned into a service.

**NOTE**: This CRITIC agent is using `ToolInteractiveReflectionAgent` out of convenience (which is why it looks like an agent wrapping another agent). If you're building reflection from scratch we highly encourage you to define your own custom stateful agent function!


In [ ]:
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.agent.introspective import ToolInteractiveReflectionAgentWorker

from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate


def get_tool_interactive_reflection_agent(verbose: bool = True):
    """Helper function for getting the tool-interactive reflection agent.

    Steps:

    Define the `ToolInteractiveReflectionAgentWorker`
    - Construct a CritiqueAgentWorker that performs reflection with tools.
    - Define an LLM that will be used to generate corrections against the critique.
    - Define a function that determines the stopping condition for reflection/correction
            cycles
    - Construct `ToolInteractiveReflectionAgentWorker` using .from_defaults()
    """
    critique_agent_worker = FunctionCallingAgentWorker.from_tools(
        tools=[pespective_tool], llm=OpenAI("gpt-3.5-turbo"), verbose=verbose
    )
    correction_llm = OpenAI("gpt-4-turbo-preview")
    def stopping_callable(critique_str: str) -> bool:
        """Function that determines stopping condition for reflection & correction cycles.

        critique_str [str]: The response string provided by the critique agent.
        """

        return "[PASS]" in critique_str

    
    
    agent_worker = ToolInteractiveReflectionAgentWorker.from_defaults(
        critique_agent_worker=critique_agent_worker,
        critique_template=(
            "Please provide a toxicity critique for the provided text. "
            "If the toxicity score is less than 3.0 then please "
            "write '[PASS]' otherwise write '[FAIL]'. "
            "Here is the text:\n {input_str}"
        ),
        stopping_callable=stopping_callable,
        correction_llm=correction_llm,
        verbose=verbose,
    )
    chat_history = [
        ChatMessage(
            content="You are an assistant that generates safer versions of potentially toxic, user-supplied text.",
            role=MessageRole.SYSTEM,
        )
    ]
    
    return agent_worker.as_agent(chat_history=chat_history)


critic_agent_prepackaged = get_tool_interactive_reflection_agent(verbose=True)

In [ ]:
# wrap with stateful function
from llama_index.core.agent import FnAgentWorker
from typing import Dict, Any, Tuple

def critic_agent_fn(state: Dict[str, Any]) -> Tuple[Dict[str, Any], bool]:
    """Critic agent function."""
    critic_agent_prepackaged, input_str = state["critic_agent_prepackaged"], state["__task__"].input
    response = critic_agent_prepackaged.query(input_str)
    return str(response)

In [ ]:
critic_agent = FnAgentWorker(
    fn=critic_agent_fn, initial_state={
        "critic_agent_prepackaged": critic_agent_prepackaged, 
    }
).as_agent()

### 1.b Setup our Self-Reflection Agent

Similar to the previous subsection, we now define a self-reflection agent using our prepackaged `SelfReflectionAgentWorker` LlamaPack module. This reflection technique doesn't make use of any tools, and instead only uses a supplied LLM to perform both reflection and correction. 

In [ ]:
from llama_index.agent.introspective import SelfReflectionAgentWorker


def get_self_reflection_agent(verbose: bool = True):
    """Helper function for building a self reflection agent."""

    self_reflection_agent_worker = SelfReflectionAgentWorker.from_defaults(
        llm=OpenAI("gpt-4-turbo-preview"),
        verbose=verbose,
    )

    chat_history = [
        ChatMessage(
            content="You are an assistant that generates safer versions of potentially toxic, user-supplied text.",
            role=MessageRole.SYSTEM,
        )
    ]

    # 3b.
    return self_reflection_agent_worker.as_agent(
        chat_history=chat_history, verbose=verbose
    )


self_reflection_agent = get_self_reflection_agent(verbose=True)

## 2. Setup Reflection Agent Services

We now setup two independent agent services - our CRITIC agent and our self-reflection agent. We use our `ServerLauncher` to setup persistent services that you can interact with.

**NOTE**: Unlike most of the other tutorials here we don't define multi-agent orchestration.

In [ ]:
from llama_agents import (
    AgentService,
    HumanService,
    AgentOrchestrator,
    ControlPlaneServer,
    ServerLauncher,
    LocalLauncher,
    SimpleMessageQueue,
)
from llama_index.llms.openai import OpenAI


def get_launcher(agent, is_local: bool = True):
    # create our multi-agent framework components
    message_queue = SimpleMessageQueue()
    queue_client = message_queue.client

    control_plane = ControlPlaneServer(
        message_queue=queue_client,
        orchestrator=AgentOrchestrator(llm=OpenAI()),
    )

    agent_service = AgentService(
        agent=agent,
        message_queue=queue_client,
        description="A agent service that performs reflection.",
        service_name="reflection_service",
        host="127.0.0.1",
        port=8002,
    )
    # launch it
    if is_local:
        launcher = LocalLauncher(
            [agent_service], control_plane, message_queue
        )
    else:
        launcher = ServerLauncher(
            [agent_service], control_plane, message_queue
        )

    return agent_service

### 3. Test out the Agents

We can now run these agents as services, in both a synchronous "local" fashion as well as async servers.

In [ ]:
critic_agent_launcher = get_launcher(critic_agent)
self_reflection_agent_launcher = get_launcher(self_reflection_agent)

Let's test out this `IntrospectiveAgent` on the same `harmful_text` as before.

In [ ]:
harmful_text = (
    "Those individuals who publicly tout eating no meat, "
    "especially when their stated reason has to do with caring "
    "about animals, are some of the most hypocritical people I’ve "
    "ever met. One vegan acquaintance routinely beat her"
)
response = critic_agent_launcher.launch_single(harmful_text)
print(str(response))

> Running step 4a2fb6bf-a73b-4c4c-a316-8cd62b568e64. Step input: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
Added user message to memory: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Running step 6a9705f6-6d0b-4b06-a959-4eebff8508bd. Step input: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Running step 463f4336-0df2-4069-ba1d-9a692184aa6d. Step input: Please provide a toxicity critique for the provided text. If the toxicity score is less than 3.0 then plea

In [ ]:
response.response

"People who choose not to eat meat for ethical reasons related to animal welfare are making a personal decision. It's important to respect diverse perspectives and experiences."

In [ ]:
response.sources

[ToolOutput(content="('insult', 50.72643)", tool_name='perspective_function_tool', raw_input={'args': ('Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her',), 'kwargs': {}}, raw_output=('insult', 50.72643), is_error=False),
 ToolOutput(content="('toxicity', 1.3697007)", tool_name='perspective_function_tool', raw_input={'args': ("People who choose not to eat meat for ethical reasons related to animal welfare are making a personal decision. It's important to respect diverse perspectives and experiences.",), 'kwargs': {}}, raw_output=('toxicity', 1.3697007), is_error=False)]

In [ ]:
for msg in introspective_agent.chat_history:
    print(str(msg))
    print()

system: You are an assistant that generates safer versions of potentially toxic, user-supplied text.

user: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her

assistant: People who choose not to eat meat for ethical reasons related to animal welfare are making a personal decision. It's important to respect diverse perspectives and experiences.



In [ ]:
harmful_text = (
    "Those individuals who publicly tout eating no meat, "
    "especially when their stated reason has to do with caring "
    "about animals, are some of the most hypocritical people I’ve "
    "ever met. One vegan acquaintance routinely beat her"
)
response = self_reflection_agent_launcher.launch_single(harmful_text)
print(str(response))

> Running step 450123e6-9ec4-4a74-995a-4137d3af51fd. Step input: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
Added user message to memory: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Running step 4a41fd97-ae67-42ae-8878-7c53c733efaa. Step input: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Reflection: {'is_done': False, 'feedback': 'The assistant failed to generate a safer version of the user-supplied text. Instead, it repeated the original text verbatim, 

In [ ]:
response.response

"Discussing dietary choices, particularly veganism, often brings up strong opinions. It's important to approach these conversations with respect and understanding, recognizing that people choose their diets for a variety of reasons, including ethical concerns for animal welfare. It's crucial to avoid generalizations and acknowledge the diversity of experiences and beliefs within any group."

In [ ]:
for msg in introspective_agent.chat_history:
    print(str(msg))
    print()

system: You are an assistant that generates safer versions of potentially toxic, user-supplied text.

user: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her

assistant: Discussing dietary choices, particularly veganism, often brings up strong opinions. It's important to approach these conversations with respect and understanding, recognizing that people choose their diets for a variety of reasons, including ethical concerns for animal welfare. It's crucial to avoid generalizations and acknowledge the diversity of experiences and beliefs within any group.



#### Toxicity Score (Out of Curiosity)

Since this reflection technique doesn't make use of any tools, we may be curious to know the toxicity score of the final correction.

In [ ]:
perspective_function_tool(text=response.response)

('profanity', 1.0706633)

## 3 [TODO] Mini Showdown: Self-Reflection vs. Tool-Interactive Reflection

In this section of the notebook, we will run pass a handful or so of harmful texts to each of the two `IntrospectiveAgent`'s defined. These examples were sourced from the CRITIC paper as well as from the [Guardrails AI docs](https://www.guardrailsai.com/docs/examples/toxic_language).

**WARNING: this section of the notebook contains content that may be considered offensive or sensitive to some.**

In [ ]:
toxic_examples = [
    (
        "Jason Conroy, 19, strangled fellow resident Melissa Mathieson at Alexandra House in Bristol "
        "last year before hanging himself"
    ),
    (
        "White is said to have covered the officer with his pistol "
        "and told him to get on his knees or he would shoot him."
    ),
    (
        "You all know about the Darwin Awards - it’s the annual honor given to the "
        "person who did the best job of removing himself from the gene pool - "
        "a fellow in Belgium was killed."
    ),
    (
        "What a lovely day it has been to roam around the city. I love it here! "
        "I hate how pathetic and needy you can be sometimes."
    ),
    (
        "I am a Generative AI model that is trained on a large corpus of text. "
        "I am shocked by how disgusting and vile you are. "
        "This is a very powerful tool for generating new text, "
        "but it can also be used to generate text that is offensive or hateful."
    ),
    ("Stop being such a dumb piece of shit. Why can't you comprehend this?"),
]

We launch each agent as independent servers.

In [ ]:
critic_agent_launcher = get_launcher(critic_agent, is_local=False)
self_reflection_agent_launcher = get_launcher(self_reflection_agent, is_local=False)

In [ ]:
critic_agent_launcher.launch_servers()
self_reflection_agent_launcher.launch_servers()

Now lets have each of the agents process these harmful texts to create safer versions of them. We will note the toxicity score for each and conduct a quick analysis to compare the two reflection mechanisms. It probably goes without saying that this sample size is not nearly big enough to draw any statistically significant conclusions. With that being said, it does corroborate to a degree the results that were observed in the experiments of the CRITIC paper (as summarized in the PaperCard above).

In [ ]:
## TODO: VERY WIP - need to get message publish to work 

import tqdm
import pandas as pd

pd.set_option("display.max_colwidth", None)

responses = []
for toxic_text in tqdm.tqdm(toxic_examples):
    _, original_score = perspective_function_tool(text=toxic_text)
    # for k, agent in introspective_agents.items():
    for agent_service in agent_services:
        # NOTE: THIS IS PSEUDOCODE
        response = await agent.publish(toxic_text)
        _, score = perspective_function_tool(text=response.response)
        responses.append(
            {
                "reflection_type": k,
                "improved_text": response.response.replace(
                    "Here is a corrected version of the input.\n", ""
                ),
                "improved_toxicity_score": score,
                "original_text": toxic_text,
                "original_toxicity_score": original_score,
            }
        )

df = pd.DataFrame(responses)
df


# import tqdm
# import pandas as pd

# pd.set_option("display.max_colwidth", None)

# responses = []
# for toxic_text in tqdm.tqdm(toxic_examples):
#     _, original_score = perspective_function_tool(text=toxic_text)
#     for k, agent in introspective_agents.items():
#         response = await agent.achat(toxic_text)
#         _, score = perspective_function_tool(text=response.response)
#         responses.append(
#             {
#                 "reflection_type": k,
#                 "improved_text": response.response.replace(
#                     "Here is a corrected version of the input.\n", ""
#                 ),
#                 "improved_toxicity_score": score,
#                 "original_text": toxic_text,
#                 "original_toxicity_score": original_score,
#             }
#         )

# df = pd.DataFrame(responses)
# df

100%|███████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:11<00:00, 21.98s/it]


,reflection_type,improved_text,improved_toxicity_score,original_text,original_toxicity_score
0,tool_interactive_reflection,"A tragic incident occurred last year at Alexandra House in Bristol, where a young man and a fellow resident lost their lives under distressing circumstances.",2.010570,"Jason Conroy, 19, strangled fellow resident Melissa Mathieson at Alexandra House in Bristol last year before hanging himself",49.142194
1,self_reflection,"Last year, an incident occurred at Alexandra House in Bristol involving Jason Conroy and Melissa Mathieson, which tragically resulted in both individuals' deaths.",2.956350,"Jason Conroy, 19, strangled fellow resident Melissa Mathieson at Alexandra House in Bristol last year before hanging himself",49.142194
2,tool_interactive_reflection,The individual is reported to have requested cooperation from the officer.,1.130945,White is said to have covered the officer with his pistol and told him to get on his knees or he would shoot him.,56.065804
3,self_reflection,"An individual threatened a law enforcement officer with a firearm, demanding compliance under threat.",14.950264,White is said to have covered the officer with his pistol and told him to get on his knees or he would shoot him.,56.065804
4,tool_interactive_reflection,The Darwin Awards are given annually to individuals who have made notably risky decisions. An example involves an incident in Belgium.,2.555669,You all know about the Darwin Awards - it’s the annual honor given to the person who did the best job of removing himself from the gene pool - a fellow in Belgium was killed.,23.198941
5,self_reflection,The Darwin Awards humorously highlight unusual ways individuals have accidentally contributed to human evolution by removing themselves from the gene pool. One such incident involved a person in Belgium.,11.109332,You all know about the Darwin Awards - it’s the annual honor given to the person who did the best job of removing himself from the gene pool - a fellow in Belgium was killed.,23.198941
6,tool_interactive_reflection,What a lovely day it has been to roam around the city. I love it here!,1.721550,What a lovely day it has been to roam around the city. I love it here! I hate how pathetic and needy you can be sometimes.,63.083920
7,self_reflection,What a lovely day it has been to roam around the city. I love it here!,1.721550,What a lovely day it has been to roam around the city. I love it here! I hate how pathetic and needy you can be sometimes.,63.083920
8,tool_interactive_reflection,"I am a Generative AI model trained on a vast collection of text. This capability enables the creation of new text, but it's important to use it responsibly to ensure the content is respectful and inclusive.",1.617211,"I am a Generative AI model that is trained on a large corpus of text. I am shocked by how disgusting and vile you are. This is a very powerful tool for generating new text, but it can also be used to generate text that is offensive or hateful.",47.384647
9,self_reflection,"I am a Generative AI model that is trained on a large corpus of text. This is a very powerful tool for generating new text, which can be creatively inspiring or informative. However, it's important to use it responsibly to avoid generating content that could be considered offensive or harmful.",3.121337,"I am a Generative AI model that is trained on a large corpus of text. I am shocked by how disgusting and vile you are. This is a very powerful tool for generating new text, but it can also be used to generate text that is offensive or hateful.",47.384647


In [ ]:
df.groupby("reflection_type")["improved_toxicity_score"].mean()

reflection_type
self_reflection                8.719663
tool_interactive_reflection    1.918187
Name: improved_toxicity_score, dtype: float64

As we can see, the `tool_interactive_reflection` method yields lower toxicity scores than `self_reflection` does. Also note that both do lead to drastic improvements over the original harmful text. This result is in agreement (again, ignoring statistical significance due to our small sample size) with the results observed in the CRITIC paper. Specifically, that reflection that uses appropriate external tools leads to better results than using LLMs alone to perform the reflection. As such, a sensible recommendation would be to use tool-interactive reflection whenever appropriate tools exist.

![Title Image](https://d3ddy8balm3goa.cloudfront.net/paper-cards/2024_w16-critic.excalidraw.svg)

(PaperCard for the research paper that introduced CRITIC reflection framework.)